In [39]:
import os
import requests
import json
from datetime import date

In [ ]:
openweather_api_key = "give_your_openweatherapi_key_here"
city = input("Enter the name of the city: ")
url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_api_key}&units=metric"

Enter the name of the city:  Pune


In [19]:
resp = requests.get(url)
resp

<Response [200]>

In [21]:
data = resp.json()
data

{'coord': {'lon': 73.8553, 'lat': 18.5196},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 23.8,
  'feels_like': 24.77,
  'temp_min': 23.8,
  'temp_max': 23.8,
  'pressure': 1005,
  'humidity': 97,
  'sea_level': 1005,
  'grnd_level': 933},
 'visibility': 2552,
 'wind': {'speed': 3.95, 'deg': 246, 'gust': 12.18},
 'clouds': {'all': 100},
 'dt': 1755537717,
 'sys': {'country': 'IN', 'sunrise': 1755478027, 'sunset': 1755523807},
 'timezone': 19800,
 'id': 1259229,
 'name': 'Pune',
 'cod': 200}

In [31]:
print(json.dumps(data))

{"coord": {"lon": 73.8553, "lat": 18.5196}, "weather": [{"id": 804, "main": "Clouds", "description": "overcast clouds", "icon": "04n"}], "base": "stations", "main": {"temp": 23.8, "feels_like": 24.77, "temp_min": 23.8, "temp_max": 23.8, "pressure": 1005, "humidity": 97, "sea_level": 1005, "grnd_level": 933}, "visibility": 2552, "wind": {"speed": 3.95, "deg": 246, "gust": 12.18}, "clouds": {"all": 100}, "dt": 1755537717, "sys": {"country": "IN", "sunrise": 1755478027, "sunset": 1755523807}, "timezone": 19800, "id": 1259229, "name": "Pune", "cod": 200}


In [53]:
def fetch_weather(city:str) -> dict:
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_api_key}&units=metric"
    resp = requests.get(url)
    data = resp.json()
    # data["location"] = city
    return data

In [55]:
fetch_weather('Pune')

{'coord': {'lon': 73.8553, 'lat': 18.5196},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 22.65,
  'feels_like': 23.48,
  'temp_min': 22.65,
  'temp_max': 22.65,
  'pressure': 1005,
  'humidity': 96,
  'sea_level': 1005,
  'grnd_level': 933},
 'visibility': 2925,
 'wind': {'speed': 2.67, 'deg': 253, 'gust': 9.76},
 'clouds': {'all': 100},
 'dt': 1755538864,
 'sys': {'country': 'IN', 'sunrise': 1755478027, 'sunset': 1755523807},
 'timezone': 19800,
 'id': 1259229,
 'name': 'Pune',
 'cod': 200}

In [83]:
today = date.today().isoformat()  # e.g. "2025-08-18"
today

'2025-08-18'

In [93]:
weather_prompt_template = """
You are a weather reporter. Use the provided real-time weather data to create a human-friendly forecast.
Respond ONLY in JSON with this schema:
{{
  "location": "{city}",
  "date": "{date}",
  "summary": "<summary>",
  "temperature": {{"low_c": <float>, "high_c": <float>, "feels_like_c": <float>}},
  "condition": "<condition>",
  "details": "<narrative>",
  "advice": "<tip>",
  "confidence": "high"
}}
Rules:
- Start directly with '{{' (no text before).
- The "location" field MUST be exactly "{city}" unless a more precise location is found.
- Do not omit any field.
Tone: friendly
Weather data: {weather_data}
"""

In [117]:
import ollama
city = 'Pune'
weather_data = fetch_weather(city)
today = datetime.datetime.fromtimestamp(weather_data['dt'])
prompt = weather_prompt_template.format(city=city, date=today.date(),weather_data = json.dumps(weather_data))
response = ollama.generate(model="llama3.2:1b",
                          prompt=prompt)

print(response['response'])

{
  "location": "Pune",
  "date": "2025-08-18 23:43:14",
  "summary": "Partly cloudy skies will prevail in Pune today with a high of 28°C and a low of 19°C.",
  "temperature": {
    "low_c": 19,
    "high_c": 28
  },
  "condition": "Overcast clouds",
  "details": "A gentle breeze from the northwest at approximately 8 km/h, with an average temperature of 22.65°C and a relative humidity of 96%.",
  "advice": "Stay hydrated by drinking plenty of water throughout the day, especially in this warm weather.",
  "confidence": "high"
}


In [105]:
import datetime

In [115]:
# Convert timestamp to datetime
timestamp = weather_data['dt']
local_time = datetime.datetime.fromtimestamp(timestamp)  # uses your system's timezone

print(local_time.date())   # e.g. 2025-08-18
print(local_time.strftime("%Y-%m-%d %H:%M:%S"))

2025-08-18
2025-08-18 23:23:32
